
Aircraft Fleet Data ETL Script
-----------------------------
This script processes aircraft fleet data from multiple airlines, extracting information 
from HTML files and transforming it into structured data for analysis.



In [5]:
# Import required libraries
import pandas as pd
from bs4 import BeautifulSoup
import os
from pathlib import Path
import re
from datetime import datetime
import numpy as np

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [40]:


def fix_delivery_date(date_str):
    """Converts delivery date formats to a consistent DD Mon YYYY format."""
    if not date_str or pd.isna(date_str):
        return None  # Return None for missing values
    
    # Match 'Month Year' format (e.g., 'Jul 2023')
    month_year_match = re.match(r'([A-Za-z]+) (\d{4})', date_str)
    if month_year_match:
        month, year = month_year_match.groups()
        return f'01 {month} {year}'  # Convert to '01 Mon YYYY'
    
    # Validate and return existing 'DD Mon YYYY' format
    try:
        return datetime.strptime(date_str, '%d %b %Y').strftime('%d %b %Y')
    except ValueError:
        return None  # Return None if the format is unexpected

def clean_airline_names(name):
    """Standardizes airline names based on predefined mapping."""
    name = name.strip()
    
    airline_name_mapping = {
        "Qatar Airways": "Qatar Airways",
        "Signapore Airlines": "Singapore Airlines",
        "Emirates": "Emirates",
        "All Nippon Airways": "ANA",
        "Cathay Pacific": "Cathay Pacific",
        "Japan Airlines": "Japan Airlines",
        "Turkish Airlines": "Turkish Airlines",
        "Eva Air": "EVA Air",
        "Air France": "Air France",
        "Swiss Internaitonal Air Lines": "Swiss"
    }
    
    return airline_name_mapping.get(name, name)

def assign_airline_id(airline_name):
    """Assigns an airline ID based on a predefined mapping."""
    airline_id_mapping = {
        "Qatar Airways": 1,
        "Singapore Airlines": 2,
        "Emirates": 3,
        "ANA": 4,
        "Cathay Pacific": 5,
        "Japan Airlines": 6,
        "Turkish Airlines": 7,
        "EVA Air": 8,
        "Air France": 9,
        "Swiss": 10
    }
    return airline_id_mapping.get(airline_name)

def standardize_aircraft_type(aircraft_type):
    """Standardizes aircraft type based on predefined hierarchical rules."""
    aircraft_type = aircraft_type.strip()
    
    # Step 1: Exact matches (prioritized first)
    exact_mappings = {
        "Airbus A318-111": "Airbus A318",
        "Bombardier CSeries CS100 (BD-500-1A10)": "Bombardier CSeries CS100",
        "Bombardier CSeries CS300 (BD-500-1A11)": "Bombardier CSeries CS300",
        "Bombardier DHC-8-402Q Dash 8": "Bombardier DHC-8"
    }
    if aircraft_type in exact_mappings:
        return exact_mappings[aircraft_type]
    
    # Step 2: Conditional mappings (broader categories, avoiding conflicts)
    if "Airbus A319" in aircraft_type:
        return "Airbus A319"
    if "Airbus A320-232" in aircraft_type or "Airbus A320-214" in aircraft_type or "Airbus A320-251N" in aircraft_type or "Airbus A320-271N" in aircraft_type or "Airbus A320-233" in aircraft_type:
        return "Airbus A320"
    if "Airbus A321" in aircraft_type:
        return "Airbus A321"
    if "Airbus A330-243F" in aircraft_type or "Airbus A330-223F" in aircraft_type:
        return "Airbus A330-200F"
    if "Airbus A330-202" in aircraft_type or "Airbus A330-223" in aircraft_type or "Airbus A330-203" in aircraft_type or "Airbus A330-243" in aircraft_type:
        return "Airbus A330-200"
    if "Airbus A330-343" in aircraft_type or "Airbus A330-303" in aircraft_type or "Airbus A330-302" in aircraft_type or "Airbus A330-342" in aircraft_type:
        return "Airbus A330-300"
    if "Airbus A350-9" in aircraft_type:
        return "Airbus A350-900"
    if "Airbus A350-1" in aircraft_type:
        return "Airbus A350-1000"
    if "Airbus A380" in aircraft_type:
        return "Airbus A380"
    if "Airbus 340" in aircraft_type or "Airbus A340-313" in aircraft_type:
        return "Airbus A340-300"
    if "Boeing 737-8" in aircraft_type:
        return "Boeing 737-800"
    if "Boeing 737-9" in aircraft_type:
        return "Boeing 737-900ER"
    if "Boeing 747-8" in aircraft_type:
        return "Boeing 747-8i"
    if "Boeing 747-4" in aircraft_type:
        return "Boeing 747-400F"
    if "Boeing 767-3" in aircraft_type:
        return "Boeing 767-300ER"
    if "Boeing 777-3" in aircraft_type or any(x in aircraft_type for x in ["Boeing 777-346ER", "Boeing 777-3F2ER", "Boeing 777-36NER", "Boeing 777-3U8ER", "Boeing 777-3Q8ER", "Boeing 777-31HER", "Boeing 777-328ER", "Boeing 777-381ER", "Boeing 777-35EER", "Boeing 777-3SHER", "Boeing 777-3ALER", "Boeing 777-3DEER", "Boeing 777-312ER", "Boeing 777-367", "Boeing 777-31H"]):
        return "Boeing 777-300ER"
    if "Boeing 777F" in aircraft_type or any(x in aircraft_type for x in ["Boeing 777-FF2", "Boeing 777F", "Boeing 777F28", "Boeing 777F1H", "Boeing 777F5E", "Boeing 777-F28", "Boeing 777-F1H", "Boeing 777-F5E", "Boeing 777-F"]):
        return "Boeing 777F"
    if "Boeing 777-2DZLR" in aircraft_type or "Boeing 777-21HLR" in aircraft_type:
        return "Boeing 777-200LR"
    if "Boeing 777-281ER" in aircraft_type or "Boeing 777-281" in aircraft_type or "Boeing 777-228ER" in aircraft_type:
        return "Boeing 777-200ER"
    if "Boeing 787-8" in aircraft_type:
        return "Boeing 787-8"
    if "Boeing 787-9" in aircraft_type:
        return "Boeing 787-9"
    if "Boeing 787-10" in aircraft_type:
        return "Boeing 787-10"
    if "Airbus A220-300 (BD-500-1A11)" in aircraft_type or "Airbus A220-100 (BD-500-1A10)" in aircraft_type:
        return "Airbus A220"
    
    return aircraft_type  # Return original if no match found

def extract_column_headers(container):
    """Extract column positions dynamically from the table header."""
    header_row = container.find('div', class_='dt-tr')  # The header row is always the first div with class "dt-tr"
    headers = [header.get_text(strip=True) for header in header_row.find_all('div', class_='dt-th')]

    column_map = {
        "Registration": headers.index("Reg") if "Reg" in headers else None,
        "Aircraft_Type": headers.index("Aircraft Type") if "Aircraft Type" in headers else None,
        "Configuration": headers.index("Config") if "Config" in headers else None,
        "Delivery_Date": headers.index("Delivered") if "Delivered" in headers else None,
        "Remarks": headers.index("Remark") if "Remark" in headers else None,
        "Aircraft_Name": headers.index("Aircraft Name") if "Aircraft Name" in headers else None,
        "Age": headers.index("Age") if "Age" in headers else None,
    }
    return column_map

def extract_row_data(row, column_map, page_num):
    """Extract data from a row dynamically, ensuring correct mapping of Aircraft Type and Delivery Date."""
    cells = row.find_all('div', class_='dt-td')

    def get_cell_value(column_name):
        idx = column_map.get(column_name)
        return cells[idx].get_text(strip=True) if idx is not None and idx < len(cells) else ""

    registration = get_cell_value("Registration")

    # Extract Aircraft Type from <a> tag if available
    type_cell = cells[column_map["Aircraft_Type"]].find("a")
    aircraft_type = type_cell.text.strip() if type_cell else get_cell_value("Aircraft_Type")

    configuration = get_cell_value("Configuration")

    # Extract Delivery Date and ensure "DD MMM YYYY" format
    delivery_date_raw = get_cell_value("Delivery_Date")
    delivery_date = fix_delivery_date(delivery_date_raw)
    delivery_status = "Delivered" if delivery_date else delivery_date_raw  # Keep original status if no valid date

    remarks = get_cell_value("Remarks")
    aircraft_name = get_cell_value("Aircraft_Name")
    age = get_cell_value("Age")

    return {
        'Registration': registration,
        'Aircraft_Type': aircraft_type,
        'Configuration': configuration,
        'Delivery_Date': delivery_date,
        'Delivery_Status': delivery_status,
        'Remarks': remarks,
        'Aircraft_Name': aircraft_name,
        'Age': age
    }

def process_airline_file(file_path):
    """Process a single airline file containing multiple pages."""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    airline_name = clean_airline_names(Path(file_path).stem.replace('_', ' ').title())
    soup = BeautifulSoup(content, 'html.parser')
    fleet_containers = soup.find_all('div', class_='fleetlist__container')

    print(f"Processing {airline_name} - Found {len(fleet_containers)} pages")

    all_aircraft_data = []
    for page_num, container in enumerate(fleet_containers, 1):
        column_map = extract_column_headers(container)  # Detect columns dynamically per page
        rows = container.find_all('div', class_='dt-tr')[1:]  # Skip header row

        for row in rows:
            row_data = extract_row_data(row, column_map, page_num)
            if row_data:
                row_data['Page'] = page_num
                row_data['Airline'] = airline_name
                all_aircraft_data.append(row_data)

    df = pd.DataFrame(all_aircraft_data)
    
    # Apply standardization after all data extraction
    df['Standardized_Aircraft_Type'] = df['Aircraft_Type'].apply(standardize_aircraft_type)
    
    return df

def process_all_airlines(directory_path):
    """Process all airline HTML files in a directory."""
    all_airlines_data = []

    for file_path in Path(directory_path).glob('*.html'):
        try:
            airline_data = process_airline_file(file_path)
            all_airlines_data.append(airline_data)
            print(f"Successfully processed {file_path.stem}")
        except Exception as e:
            print(f"Error processing {file_path.stem}: {str(e)}")

    if all_airlines_data:
        return pd.concat(all_airlines_data, ignore_index=True)
    else:
        raise Exception("No data was processed successfully")

if __name__ == "__main__":
    data_dir = "airline_data"  # Update this with your directory path
    
    try:
        fleet_data = process_all_airlines(data_dir)

        print("\nFleet Data Summary:")
        print("-" * 30)
        print(f"Total aircraft: {len(fleet_data)}")
        print("\nAircraft by airline:")
        print(fleet_data['Airline'].value_counts())
        print("\nAircraft types distribution:")
        print(fleet_data['Aircraft_Type'].value_counts())

        # Handle age conversion with special cases
        def convert_age(age_str):
            if pd.isna(age_str) or age_str == "":
                return None
            if age_str.lower() == "brand new":
                return 0.0
            # Remove 'Years' and any extra whitespace, then convert to float
            try:
                return float(age_str.replace('Years', '').strip())
            except ValueError:
                return None

        # Convert Age to float and rename column
        fleet_data['Age'] = fleet_data['Age'].apply(convert_age)
        fleet_data = fleet_data.rename(columns={'Age': 'Age (Years)'})
        
        # Add Airline ID column using predefined mapping
        fleet_data['Airline ID'] = fleet_data['Airline'].apply(assign_airline_id)
        
        # Save the transformed data
        output_file = "combined_fleet_data.csv"
        fleet_data.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"\nData exported to {output_file}")

    except Exception as e:
        print(f"Error: {str(e)}")

Processing Qatar Airways - Found 5 pages
Successfully processed qatar_airways
Processing Japan Airlines - Found 3 pages
Successfully processed japan_airlines
Processing Turkish Airlines - Found 8 pages
Successfully processed turkish_airlines
Processing Air France - Found 5 pages
Successfully processed air_France
Processing Emirates - Found 6 pages
Successfully processed emirates
Processing ANA - Found 5 pages
Successfully processed all_nippon_airways
Processing EVA Air - Found 2 pages
Successfully processed EVA_Air
Processing Swiss - Found 2 pages
Successfully processed swiss_internaitonal_air_lines
Processing Singapore Airlines - Found 3 pages
Successfully processed signapore_airlines
Processing Cathay Pacific - Found 4 pages
Successfully processed cathay_pacific

Fleet Data Summary:
------------------------------
Total aircraft: 2062

Aircraft by airline:
Airline
Turkish Airlines      385
Emirates              282
ANA                   252
Qatar Airways         238
Air France        